In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.VectorSpace
import Linear(V2(..))
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices)

In [3]:
type X = ℝ
type T = ℝ
type U = ℝ
type Ðx'U = ℝ
type x · y = ℝ²

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [4]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' (U · Ðx'U) -> Shade (Colour ℝ)
colourscheme = cm . factoriseShade . (coerceShade ::  Shade' (U · Ðx'U) -> Shade' (U, Ðx'U))
 where cm :: (Shade' U, Shade' Ðx'U) -> Shade (Colour ℝ)
       cm (Shade' u du, _) = interp (Shade u $ dualNorm du :: Shade ℝ)
       Option (Just interp) = rangeOnGeodesic darkblue orange

In [5]:
μ :: LocalLinear (X · T) (U · Ðx'U) +> (U · Ðx'U, ℝ)
μ = arr.LinearFunction $
        \(LinearMap (V2 (V2 ðx'u ðx'ðx'u)
                        (V2 ðt'u ðt'ðx'u))) -> (V2 (1e-100*ðt'ðx'u) ðx'u, ðx'ðx'u - ðt'u)

deq :: DifferentialEqn (X · T) (U · Ðx'U)
deq = constLinearPDE $ μ

In [6]:
deq ((zeroV,V2 0 1):±[(V2 1 0,zeroV), (V2 0 1,zeroV), (zeroV,V2 1 0), (zeroV,V2 0 1)])

Shade' {_shade'Ctr = ex.<V2 1.0 0.0 ^+^ ey.<V2 0.0 0.0, _shade'Narrowness = spanNorm [ex.<V2 1.0 0.0 ^+^ ey.<V2 0.0 0.0,ex.<V2 0.0 1.0e10 ^+^ ey.<V2 (-1.0e10) 0.0,ex.<V2 0.0 0.0 ^+^ ey.<V2 0.0 1.0e-100]}

In [7]:
initState :: X -> (U · Ðx'U)
initState x = V2 (tanh (s * (1 - x^2)))
                    (- 2 * s / cosh (s * (1 - x^2))^2)
 where s = 5

tf :: Needle X -> Needle T -> PointsWeb (X · T) (Shade' (U · Ðx'U))
tf δx₀ δt₀ = fromWebNodes euclideanMetric
      $  [(V2 x 0, initState x|±|[V2 0.1 0, V2 0 0.1]) | x<-[-2, δx₀-2 .. 2] ]
      ++ [(V2 x t, zeroV|±|[V2 1 0, V2 0 1]) | x<-[-2, δx₀-2 .. 2], t<-[δt₀, δt₀*2 .. 2] ]

In [8]:
startSt = tf 0.21 0.19
forM_ [ iterateFilterDEqn_static IgnoreInconsistencies deq startSt ]
  $ \tfs ->
    plotWindow
       [ plotLatest [ plot (fmap colourscheme tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ] ]

![Initial state from which to start refining solution of the heat PDE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/PDE-solution-filter/HeatEqn-InitialState.png)